In [16]:
import numpy as np
import sys

sys.path.append('../')

from mandala.nodecore import Node
from mandala.nodecore import Variable
from mandala import basic_math
from mandala import get_item

In [13]:
basic_math.install_node_arithmetics()
get_item.install_node_get_item()

計算グラフは概ねできてきたので、自動微分を行うための仕組みを設計する。ここでは Add や Linear, Convolution といったレイヤを定義するが、これらのレイヤは Node の生成をサポートするのみで、計算自体は Node によって実現される。Chainer でいうところの Link は問題ないが、Function が少々悩ましい。Chainer の Function は Variable を生成するが、Variable は自分を生成した関数とその引数を覚えており、それを使って backprop を走らせる。ところが mandala の Node はあくまで計算グラフ一般の表現であり、自身を生成した関数を覚えてはいるが、えーと。ちょっと待て。

In [129]:

def _set_grad(xs):
    for x in xs:
        if not hasattr(x, 'grad') and isinstance(x, Node):
            x.grad = None


In [108]:
import numpy as np
import sys

sys.path.append('../')

from mandala.nodecore import Node
from mandala.nodecore import Variable
from mandala import basic_math
from mandala.autodiff import autodiff

autodiff.install_node_backward()


# support function
def _make_node(func, args):
    _args = []
    for arg in args:
        if not isinstance(arg, Node):
            arg = Variable(arg)
        _args.append(arg)
    return Node(func, _args)


class Add(autodiff.AutoDiff):

    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, x0, x1, gy):
        return gy, gy


def add(lhs, rhs):
    func = Add()
    out = _make_node(func, [lhs, rhs])
    return out


class Sub(autodiff.AutoDiff):

    def forward(self, x0, x1):
        y = x0 - x1
        return y
    
    def backward(self, x0, x1, gy):
        return gy, - gy


def sub(lhs, rhs):
    func = Sub()
    out = _make_node(func, [lhs, rhs])
    return out


def rsub(rhs, lhs):
    func = Sub()
    out = _make_node(func, [lhs, rhs])
    return out


class (autodiff.AutoDiff):

    def forward(self, x0, x1):
        y = 
        return y
    
    def backward(self, x0, x1, gy):
        return 


def mul(lhs, rhs):
    def _mul(a, b):
        return a * b
    out = _make_node(_mul, [lhs, rhs])
    return out


def _div(a, b):
    return a / b


def div(lhs, rhs):
    out = _make_node(_div, [lhs, rhs])
    return out


def rdiv(lhs, rhs):
    out = _make_node(_div, [rhs, lhs])
    return out


def _floordiv(a, b):
    return a // b


def floordiv(lhs, rhs):
    out = _make_node(_floordiv, [lhs, rhs])
    return out


def rfloordiv(lhs, rhs):
    out = _make_node(_floordiv, [rhs, lhs])
    return out


def _pow(a, b):
    return a ** b


def pow(lhs, rhs):
    out = _make_node(_pow, [lhs, rhs])
    return out


def rpow(lhs, rhs):
    out = _make_node(_pow, [rhs, lhs])
    return out


def neg(a):
    def _neg(a):
        return - a
    out = _make_node(_neg, [a])
    return out


def absolute(a):
    out = _make_node(pow, [a])
    return out


def _matmul(a, b):
    return a @ b


def matmul(lhs, rhs):
    out = _make_node(_matmul, [lhs, rhs])
    return out


def rmatmul(lhs, rhs):
    out = _make_node(_matmul, [rhs, lhs])
    return out


def install_node_arithmetics():
    Node.__neg__ = neg
    Node.__abs__ = absolute
    Node.__add__ = add
    Node.__radd__ = add
    Node.__sub__ = sub
    Node.__rsub__ = rsub
    Node.__mul__ = mul
    Node.__rmul__ = mul
    Node.__div__ = div
    Node.__truediv__ = div
    Node.__rdiv__ = rdiv
    Node.__rtruediv__ = rdiv
    Node.__floordiv__ = floordiv
    Node.__rfloordiv__ = rfloordiv
    Node.__pow__ = pow
    Node.__rpow__ = rpow
    Node.__matmul__ = matmul
    Node.__rmatmul__ = rmatmul


Function は必ず値 (Node ではなく) を返さなくてはならない。そのため basic_math を利用した処理を Function 内で定義すると、Node.data がまた Node になるという状態が起こる。

In [99]:
x = Variable(np.ones((3, 3), dtype=np.float32))

In [100]:
y = add(x, x)

In [101]:
y.backward(x)

In [102]:
x.grad.data

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [103]:
y.data

array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]], dtype=float32)

In [104]:
y.backward()

In [105]:
y = x + x

In [106]:
y.data

array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]], dtype=float32)

In [110]:
func = Add().forward

In [115]:
func.__self__.backward()

TypeError: backward() missing 3 required positional arguments: 'x0', 'x1', and 'gy'

In [116]:
test = Add()

In [117]:
id(test)

1972546632168

In [118]:
id(test.forward)

1972524901960

In [120]:
id(test.forward.__self__) - id(test)

0